## 의미 검색 유사도

### 데이터 준비

In [4]:
# https://huggingface.co/datasets/klue/klue/viewer/mrc
from datasets import load_dataset

klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

In [5]:
klue_mrc_dataset

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})

In [6]:
# 일부 데이터만 사용 약 1000행 정도
data_size = 1000
klue_mrc_dataset_train = klue_mrc_dataset.train_test_split(train_size=data_size, 
                                                    shuffle=False)['train']
klue_mrc_dataset_train

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 1000
})

In [9]:
klue_mrc_dataset_train[10]
klue_mrc_dataset_train.shape

(1000, 9)

### 데이터 임베딩

In [11]:
from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model_sentence

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [12]:
embeddings = model_sentence.encode(klue_mrc_dataset_train['context'])
embeddings

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

array([[ 0.5222372 , -1.1391715 ,  0.12654728, ..., -1.1193498 ,
         0.03521844, -0.6324905 ],
       [-0.36406684, -0.6319616 ,  0.19000682, ..., -0.21136579,
         0.4021174 ,  0.38462663],
       [-0.36406684, -0.6319616 ,  0.19000682, ..., -0.21136579,
         0.4021174 ,  0.38462663],
       ...,
       [-0.92905575, -0.72629637, -0.37646997, ..., -0.05437021,
         0.72452426, -0.45626414],
       [-0.9430958 , -1.0330368 ,  0.25581682, ..., -0.24693918,
         0.28673935, -0.35674646],
       [-0.8397217 , -0.68244684, -0.07826207, ..., -0.31085145,
        -0.05979928, -0.7542277 ]], dtype=float32)

In [17]:
embeddings.shape

(1000, 768)

### 검색 인덱스 생성 (KNN)

In [14]:
!pip install faiss-cpu faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 61.8 MB/s eta 0:00:00:00:0100:01


In [21]:
import faiss # meta api : vector distance 

index_KNN = faiss.IndexFlatL2(embeddings.shape[1]) # KNN 알고리즘 초기화, vocab 사이즈 만큼 vector 공간 할당
index_KNN

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x79b7330af660> >

In [22]:
# 인덱스에 embedding 저장, 일종의 테이블 생성(메모리용 vectorDB 유사)
index_KNN.add(embeddings)

In [24]:
type(index_KNN),index_KNN

(faiss.swigfaiss_avx2.IndexFlatL2,
 <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x79b7330af660> >)

In [28]:
## 활용 
query = '이번 연도에는 언제 비가 많이 올까?'

embedding_query = model_sentence.encode([query])
embedding_query.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 768)

In [33]:
distances, indices = index_KNN.search(embedding_query, 3)
distances, indices

(array([[300.91895, 420.8628 , 420.8628 ]], dtype=float32),
 array([[  0, 920, 921]]))

In [32]:
for idx in indices[0]:
    print(klue_mrc_dataset_train['context'][idx])
    print('--------------------------------')

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.
--------------------------------
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 중에서도 태평양먹장어(Eptatretus stoutii)와 구조가 비슷하다. 게다가 오리너구리의 눈에서는 같은 포유류에게서 찾아보기 힘든 이중 원추세포가 발견된다. 

크게 발달하지도 않았고 물 속에서도 시야 확보를 위하여 쓰이지 않는 눈이지만, 오리너구리의 조상뻘 되는 종들이 한때 시각에 보다 크게 의존했다는 증거는 많이 남아 있다